In [12]:
from vnstock import *
import pandas as pd
import numpy as np
import re
import datetime
from functools import reduce
# from math import *

In [14]:
stock_history_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Stock History.csv")
income_statement_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Income Statement.csv")
balance_sheet_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Balance Sheet.csv")
cash_flow_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Cash Flow.csv")
listing_companies_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Listing Companies.csv")
financial_ratio_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Financial Ratio.csv")
stock_intraday_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Stock Intraday Transaction.csv")

general_rating_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/General Rating.csv")
business_model_rating_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Business Model Rating.csv")
business_operation_rating_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Business Operation Rating.csv")
financial_health_rating_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Financial Health Rating.csv")
valuation_rating_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Valuation Rating.csv")
industry_financial_health_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Industry Financial Health.csv")

# Dag generator

In [17]:
def dag_generator(df, eq, name, period):

    #Imports and default arguments
    script = """
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import timedelta
from vnstock import *
import pendulum
from math import *
import pandas as pd
import numpy as np  

default_args = {
'owner': 'jazzdung',
'retries':5,
'retry_delay': timedelta(minutes=20)
}

    """

    #calculate equation
    result = df.eval(eq)
    result = result.to_frame()
    result.rename(columns={0: 'result'}, inplace=True)
    result.head()



    #Write to py file
    with open("/home/jazzdung/Project III/Airflow/script/python/" + name + ".py", "w+") as dag:
        dag.write(script)
    
    #Write result to csv file (Just for now)
    result.to_csv('/home/jazzdung/Project III/Airflow/data/airflow/' + name + '.csv', sep=',', encoding='utf-8')

    return result

In [73]:
#Get list of dataframe
full_indicator_list = [income_statement_table, balance_sheet_table, cash_flow_table, financial_ratio_table]

#Join dataframes
full_indicator = reduce(lambda left, right: pd.merge(left, right, on=['ticker', 'year', 'quarter'], how = 'outer'), full_indicator_list)

# for col in full_indicator.columns:
#     print(col)

full_indicator = full_indicator.sort_values(by=['ticker', 'year','quarter'], ascending=[True, False, False])
full_indicator

,ticker,year,quarter,revenue,yearRevenueGrowth,quarterRevenueGrowth,costOfGoodSold,grossProfit,operationExpense,operationProfit,...,loanOnAsset,loanOnDeposit,depositOnEarnAsset,badDebtOnAsset,liquidityOnLiability,payableOnEquity,cancelDebt,ebitdaOnStockChange,bookValuePerShareChange,creditGrowth
70778,A32,2021,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.2,NaN,-0.221,-0.016,NaN
70886,A32,2020,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,-0.086,0.083,NaN
71038,A32,2019,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.9,NaN,0.211,0.114,NaN
71182,A32,2018,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.3,NaN,0.314,0.139,NaN
1131,AAA,2022,3,3280.0,-0.037,-0.283,-2972.0,308.0,-199.0,109.0,...,NaN,NaN,NaN,NaN,NaN,0.8,NaN,-0.076,0.152,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62590,YTC,2008,5,249.0,0.109,NaN,-226.0,23.0,-17.0,6.0,...,NaN,NaN,NaN,NaN,NaN,5.2,NaN,0.450,0.015,NaN
64648,YTC,2007,5,225.0,0.318,NaN,-209.0,16.0,-12.0,4.0,...,NaN,NaN,NaN,NaN,NaN,3.6,NaN,-0.036,0.004,NaN
66316,YTC,2006,5,171.0,-0.197,NaN,-157.0,13.0,-9.0,4.0,...,NaN,NaN,NaN,NaN,NaN,3.5,NaN,1.524,-0.019,NaN
67727,YTC,2005,5,212.0,-0.652,NaN,-200.0,13.0,-11.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
#Get list of dataframe
full_rating_list = [general_rating_table, business_model_rating_table, business_operation_rating_table, financial_health_rating_table, valuation_rating_table, industry_financial_health_table]

#Join dataframes
full_rating = reduce(lambda left, right: pd.merge(left, right, on=['ticker'], suffixes=('', '_delme'), how = 'outer'), full_rating_list)

#Remove duplicate columns
full_rating = full_rating[[c for c in full_rating.columns if not c.endswith('_delme')]]

# for col in full_rating.columns:
#     print(col)

full_rating = full_rating.sort_values(by=['ticker'], ascending=True)
full_rating.head()

,ticker,stockRating,valuation,financialHealth,businessModel,businessOperation,rsRating,taScore,highestPrice,lowestPrice,...,netDebtEquity,currentRatio,quickRatio,interestCoverage,netDebtEBITDA,pe,pb,ps,evebitda,dividendRate
0,A32,NaN,4.0,2.6,3.5,2.1,NaN,NaN,34031.0,24314.7,...,5.0,3.0,3.0,1.0,1.0,4.0,4.0,5.0,5.0,1.0
1,AAA,2.6,3.6,4.0,3.2,2.6,1.4,1.0,22800.0,5690.0,...,4.0,4.0,4.0,4.0,4.0,3.0,5.0,5.0,3.0,2.0
2,AAM,3.3,3.6,5.0,NaN,1.1,3.4,NaN,17900.0,9290.0,...,5.0,5.0,5.0,5.0,5.0,4.0,4.0,4.0,4.0,1.0
3,AAS,2.9,4.0,5.0,2.8,2.9,1.8,1.0,35528.0,7066.0,...,5.0,5.0,5.0,5.0,5.0,5.0,4.0,3.0,5.0,1.0
4,AAT,2.9,4.0,4.8,3.0,2.8,1.5,1.0,18866.3,4800.0,...,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,1.0


In [43]:
dag_generator(balance_sheet_table, 'longAsset - fixedAsset', 'testing', None)

,result
0,29.0
1,22.0
2,10.0
3,379.0
4,27.0
...,...
67108,0.0
67109,NaN
67110,0.0
67111,5.0


In [41]:
dag_generator(full_indicator, 'longAsset - fixedAsset', 'testing', None)

,result
0,1.0
1,209.0
2,2375.0
3,306.0
4,131.0
...,...
48632,0.0
48633,NaN
48634,4.0
48635,0.0
